# VGG-like model for the CIFAR-10 dataset

In this notebook, I present a model able to reach 90 % accuracy on test set by using data augmentation and using an architecture inspired by the VGG model.

- Karen Simonyan, Andrew Zisserman, 2015, Very Deep Convolutional Networks for Large-Scale Image Recognition

In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras.callbacks import ModelCheckpoint

import time

/home/arthur/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Data Preparation

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('X shape :', x_train.shape)
print(len(x_train), 'train samples')
print(len(x_test), 'test samples')

X shape : (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
x_train_mean = np.mean(x_train, axis=0)
x_train_std = np.std(x_train, axis=0)

x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_train_mean)/x_train_std

n_y = 10
y_train = keras.utils.to_categorical(y_train, n_y)
y_test = keras.utils.to_categorical(y_test, n_y)

# Model

In [4]:
input_shape = x_train.shape[1:]
activation = 'relu'

model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', input_shape=input_shape, activation=activation))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', activation=activation))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2), strides=2))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding='same', activation=activation))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', activation=activation))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2), strides=2))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), padding='same', activation=activation))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation=activation))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2), strides=2))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3,3), padding='same', activation=activation))
model.add(BatchNormalization())
model.add(Conv2D(256, (3,3), padding='same', activation=activation))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2), strides=2))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(1024, activation=activation))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_y, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
__________

# Model training

In [5]:
optimizer = keras.optimizers.adam(lr=0.001)
model.compile(optimizer, keras.losses.categorical_crossentropy, ['accuracy'])

shift = 4/32
generator = ImageDataGenerator(rotation_range=10, width_shift_range=shift, height_shift_range=shift, 
                               horizontal_flip=True)

batch_size = 64
n_steps = x_train.shape[0]//batch_size

ckeckpoint = ModelCheckpoint('./Model_trained/model_cifar10.h5', save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=25, verbose=1)

t0 = time.time()
model.fit_generator(generator.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch=n_steps, 
                    epochs=200, validation_data=(x_test, y_test), callbacks=[ckeckpoint, early_stopping])
print('Total training time : %.3f s' %(time.time()-t0))

Epoch 1/200
781/781 [==============================] - 24s 31ms/step - loss: 1.9483 - acc: 0.3697 - val_loss: 1.5767 - val_acc: 0.4889
Epoch 2/200
781/781 [==============================] - 22s 28ms/step - loss: 1.3977 - acc: 0.5235 - val_loss: 2.9576 - val_acc: 0.4749
Epoch 3/200
781/781 [==============================] - 22s 28ms/step - loss: 1.2107 - acc: 0.5916 - val_loss: 0.9561 - val_acc: 0.6682
Epoch 4/200
781/781 [==============================] - 21s 27ms/step - loss: 1.0178 - acc: 0.6539 - val_loss: 1.1099 - val_acc: 0.6664
Epoch 5/200
781/781 [==============================] - 21s 27ms/step - loss: 0.9177 - acc: 0.6888 - val_loss: 0.8687 - val_acc: 0.7195
Epoch 6/200
781/781 [==============================] - 20s 26ms/step - loss: 0.8503 - acc: 0.7113 - val_loss: 0.8723 - val_acc: 0.7242
Epoch 7/200
781/781 [==============================] - 21s 27ms/step - loss: 0.7850 - acc: 0.7317 - val_loss: 0.7582 - val_acc: 0.7592
Epoch 8/200
781/781 [==============================] - 

781/781 [==============================] - 20s 26ms/step - loss: 0.2947 - acc: 0.8972 - val_loss: 0.3444 - val_acc: 0.8886
Epoch 62/200
781/781 [==============================] - 22s 28ms/step - loss: 0.2923 - acc: 0.8981 - val_loss: 0.4017 - val_acc: 0.8725
Epoch 63/200
781/781 [==============================] - 20s 26ms/step - loss: 0.2932 - acc: 0.8964 - val_loss: 0.3863 - val_acc: 0.8779
Epoch 64/200
781/781 [==============================] - 20s 26ms/step - loss: 0.2883 - acc: 0.8994 - val_loss: 0.3275 - val_acc: 0.8940
Epoch 65/200
781/781 [==============================] - 20s 26ms/step - loss: 0.2899 - acc: 0.9002 - val_loss: 0.3516 - val_acc: 0.8855
Epoch 66/200
781/781 [==============================] - 20s 25ms/step - loss: 0.2847 - acc: 0.9017 - val_loss: 0.3454 - val_acc: 0.8893
Epoch 67/200
781/781 [==============================] - 20s 26ms/step - loss: 0.2847 - acc: 0.9026 - val_loss: 0.3595 - val_acc: 0.8845
Epoch 68/200
781/781 [==============================] - 20s 2

# Final evaluation

In [6]:
best_model = keras.models.load_model('./Model_trained/model_cifar10.h5')
print('Best model test accuracy :', best_model.evaluate(x_test, y_test, batch_size=64)[1])

10000/10000 [==============================] - 2s 197us/step
Best model test accuracy : 0.8964
